In [1]:
#Imports
import time
import sys
sys.path.append('../../')

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


In [2]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



In [3]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

In [4]:
### Sentence detector
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

In [5]:
### Tokenizer
tokenizer = Tokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

In [6]:
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")

In [7]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell")

#checked = spell_checker.fit(tokenized).transform(tokenized)

In [8]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPruneCorpus(0) \
    .setPositiveSource("../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("../../../src/test/resources/vivekn/negative") \


In [9]:
pos = PerceptronApproach() \
    .setInputCols(["sentence", "spell"]) \
    .setOutputCol("pos")

In [10]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

In [11]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    spell_checker,
    sentiment_detector,
    pos,
    finisher
])

start = time.time()
sentiment_data = pipeline.fit(data).transform(data)
sentiment_data.show()
end = time.time()
print("Time elapsed pipeline process: " + str(end - start))

+------+--------------------+--------------------+
|itemid|                text|  finished_sentiment|
+------+--------------------+--------------------+
|     1|                 ...|    result->positive|
|     2|                 ...|    result->positive|
|     3|              omg...|    result->positive|
|     4|          .. Omga...|result->positive@...|
|     5|         i think ...|result->positive@...|
|     6|         or i jus...|    result->positive|
|     7|       Juuuuuuuuu...|    result->positive|
|     8|       Sunny Agai...|    result->positive|
|     9|      handed in m...|result->positive@...|
|    10|      hmmmm.... i...|result->positive@...|
|    11|      I must thin...|    result->positive|
|    12|      thanks to a...|    result->positive|
|    13|      this weeken...|    result->positive|
|    14|     jb isnt show...|    result->positive|
|    15|     ok thats it ...|    result->positive|
|    16|    &lt;-------- ...|result->positive@...|
|    17|    awhhe man.......|re

In [12]:
for r in sentiment_data.take(5):
    print(r)

Row(itemid=1, text='                     is so sad for my APL friend.............', finished_sentiment='result->positive')
Row(itemid=2, text='                   I missed the New Moon trailer...', finished_sentiment='result->positive')
Row(itemid=3, text='              omg its already 7:30 :O', finished_sentiment='result->positive')
Row(itemid=4, text="          .. Omgaga. Im sooo  im gunna CRy. I've been at this dentist since 11.. I was suposed 2 just get a crown put on (30mins)...", finished_sentiment='result->positive@result->positive@result->positive@result->negative')
Row(itemid=5, text='         i think mi bf is cheating on me!!!       T_T', finished_sentiment='result->positive@result->positive')


In [13]:
start = time.time()
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")
end = time.time()
print("Time elapsed in write pipelines: " + str(end - start))

Time elapsed in write pipelines: 81.64870238304138


In [14]:
start = time.time()
p = Pipeline.read().load("./ps")
pm = PipelineModel.read().load("./ms")
end = time.time()
print("Time elapsed in read pipelines: " + str(end - start))

Time elapsed in read pipelines: 5.008911848068237


In [15]:
start = time.time()
pm.transform(data).where("finished_sentiment not like '%negative%'").show()
print(pm.transform(data).count())
end = time.time()
print("Time elapsed in using loaded pipelines: " + str(end - start))

+------+--------------------+--------------------+
|itemid|                text|  finished_sentiment|
+------+--------------------+--------------------+
|     1|                 ...|    result->positive|
|     2|                 ...|    result->positive|
|     3|              omg...|    result->positive|
|     5|         i think ...|result->positive@...|
|     6|         or i jus...|    result->positive|
|     7|       Juuuuuuuuu...|    result->positive|
|     8|       Sunny Agai...|    result->positive|
|     9|      handed in m...|result->positive@...|
|    10|      hmmmm.... i...|result->positive@...|
|    11|      I must thin...|    result->positive|
|    12|      thanks to a...|    result->positive|
|    13|      this weeken...|    result->positive|
|    14|     jb isnt show...|    result->positive|
|    15|     ok thats it ...|    result->positive|
|    16|    &lt;-------- ...|result->positive@...|
|    17|    awhhe man.......|result->positive@...|
|    18|    Feeling stran...|re